In [7]:
import pandas as pd
import numpy as np
import ast
import re

In [2]:
course = pd.read_csv('courses_en.csv')
course.head(10)

,url,name,category,what_you_learn,skills,language,instructors,content
0,https://www.coursera.org/learn/-network-security,Network Security,Information Technology,NaN,"Computer Networking, Network Planning And Desi...",English,['~31081695'],Welcome to course 4 of 5 of this Specializatio...
1,https://www.coursera.org/learn/-security-princ...,Security Principles,Information Technology,NaN,"Cyber Security Policies, Data Integrity, Cyber...",English,['~31081695'],Welcome to course 1 of 5 of this Specializatio...
2,https://www.coursera.org/learn/21st-century-en...,21st Century Energy Transition: how do we make...,Physical Science And Engineering,Understand the complexity of systems supplying...,"Electric Power Systems, Environmental Policy, ...",English,['brad-hayes'],NOTE: “21 st Century Energy Transition – How d...
3,https://www.coursera.org/learn/360-vr-video-pr...,VR and 360 Video Production,Arts And Humanities,NaN,"Virtual Reality, Videography, Media Production...",English,['googlearvr'],Welcome to the Google AR & VR Virtual Reality ...
4,https://www.coursera.org/learn/3d-anatomy-phys...,Foundations of Human Anatomy and Physiology,Health,Learners will understand how body structure su...,"Vital Signs, Basic Patient Care, Anatomy, Heal...",English,"['~167016541', '~166856472', '~166856442', '~1...",This course provides a foundational understand...
5,https://www.coursera.org/learn/3d-cad-application,3D CAD Application,Physical Science And Engineering,NaN,"Computer-Aided Design, Construction, 3D Assets...",English,['sckang'],Do you want to obtain all the necessary skills...
6,https://www.coursera.org/learn/3d-cad-fundamental,3D CAD Fundamental,Physical Science And Engineering,NaN,"Software Installation, SketchUp (3D Modeling S...",English,['sckang'],There have many three-dimensional shape of the...
7,https://www.coursera.org/learn/3d-cardiovascul...,The Heart of the Matter: Exploring the Cardiov...,Health,Learners will explore the structure and functi...,"Diagnostic Tests, Vital Signs, Cardiac Dysrhyt...",English,"['~167016541', '~166856472', '~166856442', '~1...",This course offers an introductory yet in-dept...
8,https://www.coursera.org/learn/3d-character-de...,3D Character Design Workshop with Nomad Sculpt,Arts And Humanities,NaN,"Color Theory, Animation and Game Design, 3D Mo...",English,['~186501695'],I'm Dave Reed and welcome to my 3D Character W...
9,https://www.coursera.org/learn/3d-game-prop-pr...,Current Gen 3D Game Prop Production,Arts And Humanities,"Model accurate block-in, mid poly meshes\nMode...","Computer Graphics, Graphical Tools, Computer G...",English,['andrewdennis'],In this course you will learn to create a phot...


In [8]:
display(course.columns.tolist())

['url',
 'name',
 'category',
 'what_you_learn',
 'skills',
 'language',
 'instructors',
 'content']

In [9]:
display(course.dtypes)

url               object
name              object
category          object
what_you_learn    object
skills            object
language          object
instructors       object
content           object
dtype: object

In [10]:
display(course.isna().sum().sort_values(ascending=False).head(20))

what_you_learn    2406
url                  0
name                 0
category             0
skills               0
language             0
instructors          0
content              0
dtype: int64

In [11]:
desc_candidates = [c for c in course.columns if c.lower() in ("description","content","long_description","summary","course_description")]
print("Description-like columns found:", desc_candidates)
# pick the first candidate if exists
DESCRIPTION_COL = desc_candidates[0] if desc_candidates else None
print("Using DESCRIPTION_COL =", DESCRIPTION_COL)


Description-like columns found: ['content']
Using DESCRIPTION_COL = content


In [12]:
if "instructors" in course.columns:
    display(course["instructors"].astype(str).head(20))
    print("\nTop value counts (sample):")
    display(course["instructors"].value_counts(dropna=False).head(20))
else:
    print("No 'instructors' column found in the dataset.")


0                                         ['~31081695']
1                                         ['~31081695']
2                                        ['brad-hayes']
3                                        ['googlearvr']
4     ['~167016541', '~166856472', '~166856442', '~1...
5                                            ['sckang']
6                                            ['sckang']
7     ['~167016541', '~166856472', '~166856442', '~1...
8                                        ['~186501695']
9                                      ['andrewdennis']
10    ['~167016541', '~166856472', '~166856442', '~1...
11                                       ['~186501695']
12    ['~166856442', '~167016541', '~166856472', '~1...
13                                       ['~124197120']
14    ['~167016541', '~166856472', '~166856442', '~1...
15    ['~166856442', '~167016541', '~166856472', '~1...
16                                        ['~89189666']
17                                        ['~891


Top value counts (sample):


instructors
['~158375127']                    229
['microsoft']                     134
['google-cloud-training']         125
['~113266557']                    112
['~186501695']                    102
['google-career-certificates']     74
['~187262598']                     45
['board-infinity']                 35
['~126164860']                     29
['~136048342']                     28
['~123403555']                     28
['~148676017']                     26
['josephcutrone']                  25
['~30575670']                      24
['juleswhite']                     23
['ibm-skills-network']             22
['bsimon']                         17
['~136285326']                     16
['~31081695']                      16
['~141793623', '~139312198']       15
Name: count, dtype: int64

In [13]:
def safe_literal_eval(x):
    if pd.isna(x):
        return None
    if isinstance(x, list):
        return x
    try:
        # handles strings like "['microsoft']"
        return ast.literal_eval(x)
    except Exception:
        # fallback: try to split by comma if it looks like "a, b"
        s = str(x).strip()
        if s.startswith("[") and s.endswith("]"):
            s = s[1:-1]
        parts = [p.strip(" '\"") for p in re.split(r',\s*', s) if p.strip()]
        return parts if parts else None

# apply if column exists
if "instructors" in course.columns:
    course["instructors_list"] = course["instructors"].apply(safe_literal_eval)
    print("Created instructors_list column. Sample:")
    display(course[["instructors","instructors_list"]].head(10))
else:
    print("No instructors column; skipping conversion.")


Created instructors_list column. Sample:


,instructors,instructors_list
0,['~31081695'],[~31081695]
1,['~31081695'],[~31081695]
2,['brad-hayes'],[brad-hayes]
3,['googlearvr'],[googlearvr]
4,"['~167016541', '~166856472', '~166856442', '~1...","[~167016541, ~166856472, ~166856442, ~166856230]"
5,['sckang'],[sckang]
6,['sckang'],[sckang]
7,"['~167016541', '~166856472', '~166856442', '~1...","[~167016541, ~166856472, ~166856442, ~166856230]"
8,['~186501695'],[~186501695]
9,['andrewdennis'],[andrewdennis]


In [14]:
# Cell 7 — extract first instructor, normalize, and flag encoded IDs
def normalize_instructor(x):
    if x is None:
        return None
    # if list => take first
    if isinstance(x, (list, tuple)):
        val = x[0] if len(x)>0 else None
    else:
        val = x
    if pd.isna(val):
        return None
    s = str(val).strip()
    if s == "":
        return None
    # treat encoded ids starting with ~ as unknown
    if s.startswith("~"):
        return "Unknown Instructor"
    # replace hyphens/underscores, collapse spaces, title-case (but keep acronym-ish words)
    s = s.replace("_"," ").replace("-"," ").strip()
    # simple smart title: keep all uppercase words as-is, else title-case
    words = []
    for w in s.split():
        words.append(w if w.isupper() and len(w)>1 else w.title())
    return " ".join(words)

if "instructors_list" in course.columns:
    course["instructor"] = course["instructors_list"].apply(normalize_instructor)
    # replace literal 'None' strings with None
    course["instructor"].replace({"None": None}, inplace=True)
    print("New 'instructor' column sample:")
    display(course[["instructors_list","instructor"]].head(10))
else:
    print("instructors_list not found — cannot create instructor column.")


New 'instructor' column sample:


C:\Users\SuperUser\AppData\Local\Temp\ipykernel_9228\378485581.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  course["instructor"].replace({"None": None}, inplace=True)


,instructors_list,instructor
0,[~31081695],Unknown Instructor
1,[~31081695],Unknown Instructor
2,[brad-hayes],Brad Hayes
3,[googlearvr],Googlearvr
4,"[~167016541, ~166856472, ~166856442, ~166856230]",Unknown Instructor
5,[sckang],Sckang
6,[sckang],Sckang
7,"[~167016541, ~166856472, ~166856442, ~166856230]",Unknown Instructor
8,[~186501695],Unknown Instructor
9,[andrewdennis],Andrewdennis


In [15]:
# Cell 8 — drop old raw instructor fields if you want
# Keep a backup by renaming instead of deleting if you prefer.
cols_to_drop = [c for c in ("instructors","instructors_list") if c in course.columns]
print("Dropping columns:", cols_to_drop)
course.drop(columns=cols_to_drop, inplace=True, errors="ignore")
print("Columns now:", course.columns.tolist())


Dropping columns: ['instructors', 'instructors_list']
Columns now: ['url', 'name', 'category', 'what_you_learn', 'skills', 'language', 'content', 'instructor']


In [17]:
# Cell 9 — fill missing what_you_learn using DESCRIPTION_COL fallback to 'content'
target_col = "what_you_learn"
desc_col = DESCRIPTION_COL
if desc_col is None:
    print("No description-like column found earlier. The filling will use 'content' if present.")
    desc_col = "content" if "content" in course.columns else None

# keywords denoting learning outcomes
keywords = ["learn","understand","build","create","master","use","apply","develop","implement","practice","explore","perform"]

def split_sentences(text):
    if pd.isna(text):
        return []
    # basic sentence split
    s = re.sub(r'\s+', ' ', str(text))
    return [seg.strip() for seg in re.split(r'(?<=[.!?])\s+', s) if seg.strip()]

def extract_learning_from_text(text):
    if pd.isna(text) or str(text).strip()=="":
        return None
    sentences = split_sentences(text)
    extracted = []
    for sent in sentences:
        s_low = sent.lower()
        if any(k in s_low for k in keywords):
            # remove trailing punctuation for neatness
            extracted.append(sent.rstrip(".!?").strip())
    # if nothing extracted, fallback to the first 1-2 sentences (useful summary)
    if len(extracted)==0 and len(sentences)>0:
        take = sentences[:2]
        take = [t.rstrip(".!?").strip() for t in take]
        return " | ".join(take)
    return " | ".join(extracted) if extracted else None

if target_col not in course.columns:
    course[target_col] = None

# Fill missing only
missing_mask = course[target_col].isna() | (course[target_col].astype(str).str.strip()=="")
print("Missing what_you_learn before:", missing_mask.sum())

if desc_col and desc_col in course.columns:
    course.loc[missing_mask, target_col] = course.loc[missing_mask, desc_col].apply(extract_learning_from_text)
else:
    print("No description column available; no automatic filling performed.")

print("Missing what_you_learn after:", course[target_col].isna().sum())
# quick preview of newly filled rows
display(course.loc[ course[target_col].notna(), ["name", desc_col, target_col ] ].head(10))


Missing what_you_learn before: 2406
Missing what_you_learn after: 0


,name,content,what_you_learn
0,Network Security,Welcome to course 4 of 5 of this Specializatio...,While most see computer networking as a positi...
1,Security Principles,Welcome to course 1 of 5 of this Specializatio...,- Relate risk management to personal or profes...
2,21st Century Energy Transition: how do we make...,NOTE: “21 st Century Energy Transition – How d...,Understand the complexity of systems supplying...
3,VR and 360 Video Production,Welcome to the Google AR & VR Virtual Reality ...,Our mission is to give you the skills you need...
4,Foundations of Human Anatomy and Physiology,This course provides a foundational understand...,Learners will understand how body structure su...
5,3D CAD Application,Do you want to obtain all the necessary skills...,Do you want to know how an engineer develop a ...
6,3D CAD Fundamental,There have many three-dimensional shape of the...,3D modeling technology can help us construct t...
7,The Heart of the Matter: Exploring the Cardiov...,This course offers an introductory yet in-dept...,Learners will explore the structure and functi...
8,3D Character Design Workshop with Nomad Sculpt,I'm Dave Reed and welcome to my 3D Character W...,The ability to design characters in 3D has bee...
9,Current Gen 3D Game Prop Production,In this course you will learn to create a phot...,"Model accurate block-in, mid poly meshes\nMode..."


In [18]:
# Cell 10 — checks
print("Total rows:", len(course))
print("what_you_learn missing:", course["what_you_learn"].isna().sum())
print("Unique instructors (non-null):", course["instructor"].nunique(dropna=True))
print("Top instructors:")
display(course["instructor"].value_counts().head(20))


Total rows: 5411
what_you_learn missing: 0
Unique instructors (non-null): 1184
Top instructors:


instructor
Unknown Instructor            2789
Microsoft                      134
Google Cloud Training          125
Google Career Certificates      74
Board Infinity                  39
Ravahuja                        31
Juleswhite                      27
Ibm Skills Network              26
Josephcutrone                   25
Bsimon                          17
Jzabel                          14
Daniel C Yeomans                13
Patrick Barry                   12
Drchuck                         11
Lmoroney                        11
Amaliabstephens                 10
Hafner                          10
Hanspeter Schaub                10
Adobe                            9
John Grim                        9
Name: count, dtype: int64

In [19]:
course.sample()

,url,name,category,what_you_learn,skills,language,content,instructor
2455,https://www.coursera.org/learn/ibm-exploratory...,Exploratory Data Analysis for Machine Learning,Data Science,This first course in the IBM Machine Learning ...,"Statistical Inference, Data Analysis, Data Pro...",English,This first course in the IBM Machine Learning ...,Unknown Instructor


In [20]:
course['tags'] = course['category'] + course['what_you_learn'] + course['skills'] + course['language'] + course['content'] + course['instructor']

In [22]:
course.sample()

,url,name,category,what_you_learn,skills,language,content,instructor,tags
305,https://www.coursera.org/learn/android-app,Build Your First Android App (Project-Centered...,Computer Science,What you’ll achieve:In this project-centered c...,"Android Studio, User Interface (UI) Design, Ap...",English,What you’ll achieve:In this project-centered c...,Galtier,Computer ScienceWhat you’ll achieve:In this pr...


In [23]:
new_df = course[['url', 'name', 'tags']]

In [24]:
new_df.sample()

,url,name,tags
1340,https://www.coursera.org/learn/design-basics-o...,Design Basics of Steel Buildings,Physical Science And EngineeringRecall foundat...


In [34]:
import nltk

In [35]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [36]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [37]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_9228\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [25]:
new_df['tags'][0]

'Information TechnologyWhile most see computer networking as a positive, criminals routinely use the internet, and the networking protocols themselves, as weapons and tools to exploit vulnerabilities and for this reason we must do our best to secure the network | We will review the basic components of a network, threats and attacks to the network, and learn how to protect them from attackers | Network security itself can be a specialty career within cybersecurity; however, all information security professionals need to understand how networks operate and are exploited to better secure them | - Discuss common tools used to identify and prevent threats | - Practice the terminology of and review network security conceptsComputer Networking, Network Planning And Design, TCP/IP, Threat Detection, Cyber Attacks, General Networking, Encryption, Threat Management, Network Protocols, Firewall, Cloud Computing, Network Security, Cybersecurity, Data Centers, Infrastructure SecurityEnglishWelcome 

In [26]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_9228\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [27]:
new_df['tags'][0]

'information technologywhile most see computer networking as a positive, criminals routinely use the internet, and the networking protocols themselves, as weapons and tools to exploit vulnerabilities and for this reason we must do our best to secure the network | we will review the basic components of a network, threats and attacks to the network, and learn how to protect them from attackers | network security itself can be a specialty career within cybersecurity; however, all information security professionals need to understand how networks operate and are exploited to better secure them | - discuss common tools used to identify and prevent threats | - practice the terminology of and review network security conceptscomputer networking, network planning and design, tcp/ip, threat detection, cyber attacks, general networking, encryption, threat management, network protocols, firewall, cloud computing, network security, cybersecurity, data centers, infrastructure securityenglishwelcome 

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5500, stop_words='english')

In [39]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [40]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(5500,))

In [44]:
cv.get_feature_names_out()

array(['00', '000', '10', ..., 'zero', 'zone', 'zoom'],
      shape=(5500,), dtype=object)

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
similarity = cosine_similarity(vectors)

In [48]:
similarity

array([[1.        , 0.38785889, 0.02505748, ..., 0.08080537, 0.11613625,
        0.04623849],
       [0.38785889, 1.        , 0.03952364, ..., 0.1008442 , 0.12164333,
        0.07245927],
       [0.02505748, 0.03952364, 1.        , ..., 0.0685454 , 0.04062507,
        0.10370039],
       ...,
       [0.08080537, 0.1008442 , 0.0685454 , ..., 1.        , 0.12049835,
        0.07841696],
       [0.11613625, 0.12164333, 0.04062507, ..., 0.12049835, 1.        ,
        0.01380043],
       [0.04623849, 0.07245927, 0.10370039, ..., 0.07841696, 0.01380043,
        1.        ]], shape=(5411, 5411))

In [49]:
def recommend(course):
    course_index = new_df[new_df['name'] == course].index[0]
    distance = similarity[course_index]
    course_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]

    for i in course_list:
        print(new_df.iloc[i[0]]['name'])

In [50]:
recommend('Design Basics of Steel Buildings')

Modelling, Analysis and Design of Steel Buildings
Construction aspects of Steel Buildings
Structural scheme setting and ETABS Analysis of RCC Building
Design Basics of RCC Buildings
Design and Detailing of RCC elements


In [51]:
import pickle

In [52]:
pickle.dump(new_df,open('course.pkl','wb'))

In [53]:
pickle.dump(similarity,open('similarity.pkl','wb'))